In [1]:
!pip install -U odp-sdk --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [45]:
from odp.client import OdpClient # The SDK
from shapely.geometry import box
import pandas as pd
import geopandas as gpd
import json
import geopandas as gpd

In [46]:
client = OdpClient()

In [47]:
## Request the dataset from the catalog using the UUID:
proSeas_dataset = client.catalog.get(("36630571-4b16-4c0e-b678-6f0e35e8014a"))
proSeas_dataset.metadata.display_name

'ProtectedSeas Navigator - Comprehensive Database of Marine Life Protections'

In [48]:
proSeas_data = client.table_v2(proSeas_dataset)

## Create a geographic query and search for specific MPAs

In [50]:
# Bounding box for Southern Norway
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 56
lat_max = 62
lon_min = 8
lon_max = 12
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

'POLYGON ((12 56, 12 62, 8 62, 8 56, 12 56))'

In [51]:
# Find Natura2000 MPAs within the search area
df_geo = pd.concat(proSeas_data.select(f"geometry within '{query_geometry}' AND boundary_source == 'Natura2000'").dataframes(), ignore_index=True)
df_geo.head()

,other_nets,nets,wdpa_id,designation,effective_from,category_name,site_minor_version,construction_prohibited,cast_nets,misc_gear,...,url,gillnetting,regulation_url,purse_seine_roundhaul_surrounding_nets,dredges,dredging_dumping,year_est,purpose,version_start_date,restrictions
0,NaN,3.0,555580346; 555722204,Site of Community Importance (Habitats Directi...,None,IUCN MPA,0,3.0,NaN,3.0,...,https://natura2000.eea.europa.eu/Natura2000/SD...,3.0,Council Directive 92/43/EEC|https://eur-lex.eu...,NaN,NaN,NaN,2011.0,Vangså Hede is a very important habitat. The d...,2021-08-03,"According to our research, this site either do..."
1,NaN,3.0,555580345; 555538070,Natura 2000: Site of Community Importance (Hab...,,IUCN MPA,0,3.0,NaN,3.0,...,MPA Website|https://natura2000.eea.europa.eu/N...,3.0,Council Directive 92/43/EEC|https://eur-lex.eu...,NaN,NaN,NaN,2007.0,To protect important habitat. The site's exten...,2021-08-03,"According to our research, this site either do..."
2,NaN,3.0,555533443,Site of Community Importance (Habitats Directive),,Other,0,3.0,NaN,3.0,...,Natura2000|https://natura2000.eea.europa.eu/Na...,3.0,Council Directive 92/43/EEC|https://eur-lex.eu...,NaN,NaN,NaN,2011.0,Idefjorden is one of the most species-rich and...,2021-08-03,"According to our research, this site either do..."
3,NaN,3.0,555533456,Site of Community Importance (Habitats Directive),None,Other,0,3.0,NaN,3.0,...,Natura2000|https://natura2000.eea.europa.eu/Na...,3.0,Council Directive 92/43/EEC|https://eur-lex.eu...,NaN,NaN,NaN,2011.0,The island has great variation in habitat-depe...,2021-08-03,"According to our research, this site either do..."
4,NaN,3.0,555522487,Site of Community Importance (Habitats Directive),None,IUCN MPA,0,3.0,NaN,3.0,...,https://natura2000.eea.europa.eu/Natura2000/SD...,3.0,Council Directive 92/43/EEC|https://eur-lex.eu...,NaN,NaN,NaN,2011.0,To protect important habitat. The area is of g...,2021-08-03,"According to our research, this site either do..."


## Get all the data from a country

In [53]:
country = "Norway" # select a country

In [54]:
df_country = pd.concat(proSeas_data.select(f"country == '{country}'").dataframes(), ignore_index=True)
df_country.head()

,other_nets,nets,wdpa_id,designation,effective_from,category_name,site_minor_version,construction_prohibited,cast_nets,misc_gear,...,url,gillnetting,regulation_url,purse_seine_roundhaul_surrounding_nets,dredges,dredging_dumping,year_est,purpose,version_start_date,restrictions
0,NaN,3.0,156009,National Park,None,IUCN MPA,0,3.0,NaN,3.0,...,https://faktaark.naturbase.no/?id=VV00003264,3.0,https://lovdata.no/forskrift/2016-12-16-1630,NaN,NaN,NaN,2016.0,The purpose of the Jomfruland National Park is...,2023-01-01,In the national park no lasting impact on natu...
1,NaN,3.0,555625764; 555595979,Marine Protected Area,,IUCN MPA,0,1.0,NaN,3.0,...,MPA Website|https://faktaark.naturbase.no/?id=...,3.0,Regulations Text|https://lovdata.no/forskrift/...,NaN,NaN,NaN,2016.0,The purpose of the Gaulosen marine conservatio...,2021-08-03,In the marine conservation area no one shall d...
2,NaN,3.0,555625765; 555595980,Marine Protected Area,,IUCN MPA,0,1.0,NaN,3.0,...,MPA Website|https://faktaark.naturbase.no/?id=...,3.0,Regulations Text|https://lovdata.no/forskrift/...,NaN,NaN,NaN,2016.0,The purpose of the Jerkysten Marine Protection...,2021-08-03,"1. The vegetation, including seaweed, kelp and..."
3,NaN,3.0,555625766; 555595981,OSPAR Marine Protected Area,,IUCN MPA,0,1.0,NaN,3.0,...,MPA Website|https://faktaark.naturbase.no/?id=...,3.0,Regulations Text|https://lovdata.no/forskrift/...,NaN,NaN,NaN,2016.0,The purpose of the Rødberget marine protected ...,2021-08-03,In the marine conservation area no one shall d...
4,NaN,3.0,183209,Bird Sanctuary,,IUCN MPA,0,1.0,NaN,3.0,...,MPA Website|https://faktaark.naturbase.no/?id=...,3.0,Regulations Text|https://www.lovdata.no/for/lf...,NaN,NaN,NaN,2003.0,The purpose of the conservation is to safeguar...,2021-08-03,1. Vegetation on land and in water (including ...


## Save it in your prefered file format

In [58]:
# Uncomment the line below to select the output format

# Export to CSV
df_country.to_csv('df_country.csv', index=False)

# Export to JSON
# df_country.to_json('df_country.json', orient='records', lines=True)

# Export to Parquet
# df_country.to_parquet('df_country.parquet', index=False)